## 機械学習手法を利用したロジックの実装

### 準備

機械学習手法を用いたリコメンドロジックを実装していくにあたって、今一度実行環境の情報を整理しておきます。
またここでの作業はJupyter NotebookないしはIPython Consoleを利用した対話的操作による実行を想定しています。

#### サービスの確認

次のコマンドをターミナルから実行し、現在の稼働しているコンテナについての情報を取得しておきます。
これらの値はユーザの実行環境ごとにことなるため、それぞれの設定についてそれぞれのプログラムの設定を書き換えるようにしてください。

```
$ docker-compose ps
    Name                  Command               State            Ports          
--------------------------------------------------------------------------------
apps_api_1     hug -f app.py                    Up      0.0.0.0:32769->8000/tcp 
apps_mongo_1   docker-entrypoint.sh mongod      Up      0.0.0.0:32768->27017/tcp
apps_ngrok_1   /entrypoint.sh                   Up      0.0.0.0:4040->4040/tcp  
apps_web_1     flask run --host 0.0.0.0 - ...   Up      0.0.0.0:80->80/tcp    
```

#### データの準備

`web`コンテナの中から機械学習を利用するためのログデータを、手元の環境にコピーしておきましょう。
次のコマンドをターミナルから実行し、ログデータを取得します。

```sh
$ cd apps/
$ mkdir -p data/
$ docker cp $(docker-compose ps -q web):/app/data/event.jsonl ./data/event.$(date +%Y-%m-%d).jsonl
$ ls data/
event.2019-02-19.jsonl
$ wc data/event.2019-02-19.jsonl
   3829  177324 2369199 data/event.2019-02-19.jsonl
```

## データセットの構築

機械学習に利用するデータセットを準備していきましょう。

サービスに利用しているMongoDBからは、機械学習のために利用する書籍に関する情報のデータセットを
ログに関するデータからユーザのサービス上の振る舞いについてのデータセットが構築できます。

#### MongoDBから書籍情報を取得する

あらためてサービスで利用している書籍情報をMongoDBより取得し、操作の行いやすいデータフレームに保持します。

In [110]:
import pandas as pd
import pymongo

# *
# 以下の情報はdocker-composeの設定より各自書き換える
# *  
mongo_authority = "0.0.0.0:32807" 

client = pymongo.MongoClient(f"mongodb://{mongo_authority}")
db = client["mynavi"]

df_books = pd.DataFrame(list(db["books"].find()))
df_books._id = df_books._id.astype(str)
df_books = df_books.rename({"_id": "record_id"}, axis=1)
df_books = df_books.set_index("record_id")

df_books.publication_day = pd.to_datetime(df_books.publication_day)
df_books.crawled_at = pd.to_datetime(df_books.crawled_at, unit='ms')

required_columns = ['author', 'body', 'genre', 'price', 'publication_day', 'title']
df_books = df_books[required_columns]

df_books.describe(include='all')

,author,body,genre,price,publication_day,title
count,1406,1406,1406,1086.000000,1347,1406
unique,401,1390,11,NaN,805,1405
top,,,将棋,NaN,2013-04-01 00:00:00,マイナビBEST 天頂の囲碁
freq,486,13,577,NaN,8,2
first,NaN,NaN,NaN,NaN,2003-06-19 00:00:00,NaN
last,NaN,NaN,NaN,NaN,2019-04-30 00:00:00,NaN
mean,NaN,NaN,NaN,1799.273481,NaN,NaN
std,NaN,NaN,NaN,1519.928550,NaN,NaN
min,NaN,NaN,NaN,473.000000,NaN,NaN
25%,NaN,NaN,NaN,1231.000000,NaN,NaN


#### ログデータを読み込む

データフレームへのログ読み込みを行います。

In [111]:
import pandas as pd

# ログデータの読み込み
_df = pd.read_json('./apps/data/event.2019-02-19.jsonl', lines=True)
_df = _df.join(_df.context.apply(pd.Series))
_df.rename({'book_id': 'record_id'}, axis=1,inplace=True)
df_logs = _df[[
    "event",
    "target",
    "user_id",
    "record_id",
    "neighbors",
    "position"
]]

df_logs.drop('neighbors', axis=1).describe(include='all')

,event,target,user_id,record_id,position
count,3829,3829,3829,2905,2905.000000
unique,3,5,99,1161,NaN
top,load,DIV,c23b213a-d813-4ecf-aee9-aaafdada76af,5c7d6b299c22452f2509fb43,NaN
freq,2773,2783,388,184,NaN
mean,NaN,NaN,NaN,NaN,0.984509
std,NaN,NaN,NaN,NaN,0.816560
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,2.000000


### 機械学習用のデータセットを構築する

#### クリック率予測用のデータセットの構築

書籍情報からクリック率を予測するモデルのためのデータセットを構築します。

In [112]:
# 各itemnのクリック数, ロード数, クリック率を算出する
g = df_logs.groupby(["event", "target", "record_id"])
df_agg = g.size()

df_data = df_agg.loc[("click", "A")].to_frame("click")
df_data = df_data.join(df_agg.loc[("load", "DIV")].to_frame("load"), how="outer")
df_data = df_data.fillna(0)
df_data['p_click'] = df_data.click / df_data.load
df_data.describe()

,click,load,p_click
count,1161.000000,1161.000000,1161.000000
mean,0.094746,2.388458,0.025237
std,1.491810,4.420349,0.136290
min,0.000000,1.000000,0.000000
25%,0.000000,1.000000,0.000000
50%,0.000000,2.000000,0.000000
75%,0.000000,3.000000,0.000000
max,50.000000,133.000000,1.000000


In [113]:
df1 = df_books.join(df_data)
df1.load = df1.load.fillna(0)
df1.p_click = df1.p_click.fillna(0)

In [114]:
df1.columns

Index(['author', 'body', 'genre', 'price', 'publication_day', 'title', 'click',
       'load', 'p_click'],
      dtype='object')

#### クリック率予測用のデータセットの構築

２つの書籍情報からよりクリックされやすい情報を取得します。

## 機械学習用のパイプラインの構築

学習/評価のための機械学習用のパイプラインを構築していきましょう。

### テキスト処理に利用する形態素解析器のMeacbのTokenizer

以前の章で構築した`sklearn`用のパイプラインをあらためて、ここでも利用します。

### mecabのインストールの確認

テキスト処理に必要なソフトウェアとして`mecab`を再度インストールしましょう。

#### CentOS(SageMaker)上でのmecabのミドルウェアのセットアッップ

```sh
$ sudo yum update -y 
$ sudo rpm -ivh http://packages.groonga.org/centos/groonga-release-1.1.0-1.noarch.rpm
$ sudo yum install mecab mecab-devel mecab-ipadic
$ mecab-config --version
0.996
```

```sh
$ pip3 install "mecab-python3==0.7"
```

In [115]:
! mecab-config --version

0.996


In [116]:
# sklearnのパイプライ処理を行うためのMecabの実装を行う
import MeCab
from sklearn.base import BaseEstimator, TransformerMixin


class Token:
    """MeCabのトークンを保持するクラス"""
    def __init__(self, node):
        # 表層形
        self.surface = node.surface

        features = node.feature.split(",")
        # 品詞
        self.part_of_speech = features[0]
        # 基本形
        self.base_form = features[6]

    def __str__(self):
        return "{}\t{}".format(self.surface, self.part_of_speech)


class MeCabTokenize(BaseEstimator, TransformerMixin):
    """MeCabを利用して形態素解析を行うクラス"""
    def __init__(self, pos_keep_filters=[]):
        # MeCabインスタンスの生成
        self.tokenizer = MeCab.Tagger("-b 100000")
        # メモリの初期化周りでバグがあるため、一度解析することで回避
        self.tokenizer.parse("init")

        # 前処理を手軽に行えるように、品詞フィルタを作る
        self.pos_keep_filters = pos_keep_filters

    def fit(self, X, y=None):
        # scikit-learn互換のインターフェイス
        return self

    def transform(self, X):
        # scikit-learn互換のインターフェイス
        docs = []
        # 1文書ずつ処理する
        for text in X:
            words = []
            # 文書内のテキストを改行でさらに文に分ける
            for sentence in self.split_text_to_sentences(text):
                # 対象の文を分かち書き
                words.extend(self.wakati(sentence))
            # 文書はスペース区切りで追加する
            docs.append(" ".join(words))
        return docs

    def split_text_to_sentences(self, text, delimiter="。"):
        return [t for t in text.replace(delimiter, delimiter + "\n").splitlines() if t]

    def wakati(self, text):
        return [t.base_form for t in self.tokenize(text)]

    def tokenize(self, text):
        tokens = []

        node = self.tokenizer.parseToNode(text)
        node = node.next
        while node.next:
            token = Token(node)
            # 品詞フィルターを適用する
            if token.part_of_speech in self.pos_keep_filters:
                tokens.append(Token(node))
            node = node.next

        return tokens

### モデルのパイプラインの組み立て

#### データフレーム向けのTransfomerの構築

今回データセットはデータフレームを利用しています。
データセットに関する様々な処理をデータフレーム上で行いやすいように、`sklearn`で必要となる
Transfomerをここで構築していきます。

##### DFColumnSelector

`DFColumnSelector`は、入力のデータフレームから、特定のカラムを取り出すTransfomerです。

In [117]:
from sklearn.base import BaseEstimator, TransformerMixin

class DFColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X[self.column]

##### AdaptedLabelEncoder
パイプラインにカテゴリ変数を組み込む際にscikit-learnで用意されているsklearn.preprocessingは
そのままではパイプライン処理に利用できません。
そのため簡単なアダプターを用意し、パイプライン上でも使えるようにクラス定義しておきます。
これによりカテゴリ変数をうまく組み込むことができるようになります。

In [118]:
from sklearn.preprocessing import LabelEncoder

class AdaptedLabelEncoder(LabelEncoder):

    def fit_transform(self, y, *args, **kwargs):
        return super().fit_transform(y).reshape(-1, 1)

    def transform(self, y, *args, **kwargs):
        return super().transform(y).reshape(-1, 1)


### 機械学習のワークフローを組み立てる

ここまでで機械学習モデルの構築に必要なパイプラインの下準備を行うことができました。
データセットの分割等を経て、モデルを構築していきましょう。

In [133]:
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

keep_pos = ["名詞", "形容詞", "動詞"]
feature_extractor_books = make_pipeline(
    FeatureUnion(transformer_list=[
        ("text_title_feature", make_pipeline(
            DFColumnSelector(column="title"),
            MeCabTokenize(pos_keep_filters=keep_pos),
            TfidfVectorizer(min_df=3)
        )),
        ("text_body_feature", make_pipeline(
            DFColumnSelector(column="body"),
            MeCabTokenize(pos_keep_filters=keep_pos),
            TfidfVectorizer(min_df=3)
        )),
    ])
)

classifier_pipeline = make_pipeline(
    feature_extractor_books,
    LinearRegression()
)

In [152]:
# クリック率に対して予測を行う
random_seed = 45
param_grid = {
    "linearregression__fit_intercept": [True, False],
    "pipeline__featureunion__text_body_feature__tfidfvectorizer__min_df": range(1,3),
    "pipeline__featureunion__text_title_feature__tfidfvectorizer__min_df": range(1,3),

    # "logisticregression__penalty": ['l1', 'l2'],
    # "logisticregression__C": [0.1, 0.01, 0.001],
}

In [153]:
X = df1.drop("p_click", axis=1)
y = df1.p_click.astype(float)

# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/10., random_state=random_seed)

In [154]:
X_train.shape

(1265, 8)

In [155]:
clf = GridSearchCV(classifier_pipeline, param_grid, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pipeline', Pipeline(memory=None,
     steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('text_title_feature', Pipeline(memory=None,
     steps=[('dfcolumnselector', DFColumnSelector(column='title')), ('mecabtokenize', MeCabTokenize(pos_keep_filters=['名詞', '形容詞', '動詞']... ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'linearregression__fit_intercept': [True, False], 'pipeline__featureunion__text_body_feature__tfidfvectorizer__min_df': range(1, 3), 'pipeline__featureunion__text_title_feature__tfidfvectorizer__min_df': range(1, 3)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

最後にテストデータにおいてモデルの評価を行ってみましょう。
回帰の問題として考えた場合には、score関数は$R^2$決定係数により計算されます。
このあたいは正負の数値をとり1に近いほど良いスコアとなります。

In [156]:
y_score = clf.score(X_test, y_test)
y_score

-0.24396623237593576

### APIサーバへのデプロイ

ここまででクリック率を予測するという形で機械学習モデルの構築を行いました。
ここからこのモデルをサービスに組込む作業を行なっていきます。

機械学習を利用したモデルの場合、入力と出力の組み合わせを考えて
サービスとして効率の良い選択肢を考えましょう。

先ほど構築したモデルは、回帰のモデルで入力データに対してクリック率の予測するモデルです。
この予測に利用する既存のコンテンツデータは不変的であるため、保持しているコンテンツに対して
あらかじめ全てを用意しておきMongoDBからこれにアクセスすることで高速に機械学習結果を利用することができます。

予測値だけ作ってしまえば、集計処理を行いAPI化する作業と同じになります。
ここで作ったモデルは機械学習モデルでクリック率を予測するため、予測結果はクリック率になると思われたかもしれません。
クリック率を予測するモデルを導入することでえられるメリットは、クリックのないもしくは
データ自体がほとんど足りていないコンテンツに対しても予測することができる点です。
新しい本が導入された場合においても予測値をモデルに組み込むことができます。
この時の新しい本の何を参考にするかは、素性に使った情報です。ここではタイトルや説明文です。

この時の予測性能は先ほどテストデータが参考になります。

作業手順としては次の通りになります。

In [174]:
df1['p_click_pred'] = clf.predict(X) * 100
df1.p_click_pred.describe()

count    1.406000e+03
mean     2.065827e+00
std      1.215828e+01
min     -2.180208e+01
25%     -5.362827e-08
50%      9.461026e-10
75%      6.099919e-08
max      1.000000e+02
Name: p_click_pred, dtype: float64

この時学習データに利用した事例についても予測を行ってしまっていることには注意してください。
以下で生成したデータをモデルの評価用途に利用すると間違った比較を行ってしまいます。

あとはこのデータをMongoDBへ格納し、APIから利用できるようにするだけです。
次のようにしてMongoDBへの格納を進めましょう

In [175]:
from datetime import datetime
data = df1[["p_click", "p_click_pred"]]
data["created_at"] = str(datetime.now())
data.sample(3)

/home/ec2-user/anaconda3/envs/chainer_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,p_click,p_click_pred,created_at
record_id,,,
5c7d6c469c22452f2509fb61,0.5,5.000000e+01,2019-03-06 00:55:02.114882
5c7d59e99c22452f2509f94a,0.0,-1.480778e-08,2019-03-06 00:55:02.114882
5c7d71259c22452f2509fbdd,0.0,-2.291788e-08,2019-03-06 00:55:02.114882


In [194]:
import pymongo

# mongodbのホスト割り当ては実行時の環境に依存する。
# 以下のコマンドを実行し、取得を行う
# $ docker-compose port mongo 27017
mongo_authority = "0.0.0.0:32807" 

client = pymongo.MongoClient(f"mongodb://{mongo_authority}")
db = client["mynavi"]

# イチオシ用のコレクションを追加する
# すでに存在す場合には以下で削除
cname = "ml_click"
db.drop_collection(cname) 
collection = db.create_collection(cname)

records = data.reset_index().replace({'record_id': 'book_id'}).to_dict('records')
collection.insert_many(records)


最後に、APIとして公開しましょう。APIサーバに次の変更を加えます。

```python
def fetch_ml_ranked_items():
    client = _build_mongodb_client()        
    db = client["mynavi"]    
    collection = db["ml_click"]
    
    items = list(collection.find().sort([
        ("date", pymongo.DESCENDING), 
        ("count", pymongo.DESCENDING)]
    ).limit(3))
    
    return [item["book_id"] for item in items]    


@hug.get("/items")
def api_ranking_by_ml_click():
    # 各自の実行環境のMongoDB上で存在するIDを指定する
    ids = merge_serieses_keeping_order(
        fetch_ml_ranked_items(),
        fetch_mongo_items(),        
    )
    ids = list(map(str, ids))
    return {
        "data": list(ids)[:3],
        "logic": "ml_click"
    }
```

In [196]:
%%writefile ./apps/webapi/app.py
import hug
import pymongo 


def _build_mongodb_client():
    user = 'root'
    password = 'set_yours_credential'
    authority = "mongo:27017"
    client = pymongo.MongoClient(f"mongodb://{user}:{password}@{authority}")
    return client

        
def fetch_mongo_items():
    client = _build_mongodb_client()    
    
    db = client["mynavi"]    
    collection = db["books"]
    
    items = list(collection.aggregate([
        {"$project": {"_id": 1}},
        {"$sample": {"size": 3}},
    ]))                 
    return [item['_id'] for item in items]


def fetch_pickup_items():
    client = _build_mongodb_client()    
    
    db = client["mynavi"]    
    collection = db["pickup"]
    return [item["book_id"] for item in collection.find({}, {"_id": 0, "book_id": 1})]    


def merge_serieses_keeping_order(lhs_ids, rhs_ids):
    from itertools import chain
    seen = set()
    for item in chain(lhs_ids, rhs_ids):
        if item in seen:
            continue
        
        seen.add(item)
        yield item
           

@hug.get("/")
def api_example():
    return "WebAPIの開発"


@hug.get("/items", versions=1)
def api_random_ids():
    # 各自の実行環境のMongoDB上で存在するIDを指定する
    ids = fetch_mongo_items()
    ids = list(map(str, ids))
    return {
        "data": ids,
        "logic": "random"
    }


# 古いロジックにバージョン番号を与える
@hug.get("/items", versions=1)
def api_random_ids_with_pickup():
    # 各自の実行環境のMongoDB上で存在するIDを指定する
    ids = merge_serieses_keeping_order(
        fetch_pickup_items(),
        fetch_mongo_items(),        
    )
    ids = list(map(str, ids))
    return {
        "data": list(ids)[:3],
        "logic": "random w/ pickup"
    }


def fetch_ranked_items():
    client = _build_mongodb_client()        
    db = client["mynavi"]    
    collection = db["ranking_click"]
    
    items = list(collection.find().sort([
        ("date", pymongo.DESCENDING), 
        ("count", pymongo.DESCENDING)]
    ).limit(3))
    
    return [item["book_id"] for item in items]    


@hug.get("/items")
def api_ranking_by_click():
    # 各自の実行環境のMongoDB上で存在するIDを指定する
    ids = merge_serieses_keeping_order(
        fetch_ranked_items(),
        fetch_mongo_items(),        
    )
    ids = list(map(str, ids))
    return {
        "data": list(ids)[:3],
        "logic": "ranking-click"
    }


def fetch_ml_ranked_items():
    client = _build_mongodb_client()        
    db = client["mynavi"]    
    collection = db["ml_click"]
    
    items = list(collection.find().sort([
        ("date", pymongo.DESCENDING), 
        ("count", pymongo.DESCENDING)]
    ).limit(3))
    
    return [item["book_id"] for item in items]    


@hug.get("/items")
def api_ranking_by_ml_click():
    # 各自の実行環境のMongoDB上で存在するIDを指定する
    ids = merge_serieses_keeping_order(
        fetch_ml_ranked_items(),
        fetch_mongo_items(),        
    )
    ids = list(map(str, ids))
    return {
        "data": list(ids)[:3],
        "logic": "ml_click"
    }

Overwriting ./apps/webapi/app.py


## まとめ

さて、ここまでで推薦サービスづくりを行ってきました。
簡単なルールベースのロジックから集計処理、そして機械学習モデルを利用した事例までです。

サービスとしてはひとつのシステムとしてつながりましたが、個々の精度としては
ここまで作ってきたサービスとしてはまだまだ改善の余地があるでしょう。

モデルのそもそもの精度やパイプラインの自動化